In [23]:
from vnau_parser import parse_address
import pandas as pd
import re

# Demo

In [2]:
parse_address('Ngã Tư Sở Market, Đường Nguyễn Trãi, Thượng Đình Số nhà 28A, Ngõ 72 Nguyễn Trãi , , Phường Thượng Đình, Quận Thanh Xuân , Hà Nội Quận Thanh Xuân')

AdministrativeUnit(province=Hà Nội, district=Thanh Xuân, province_english=Ha Noi, district_english=Thanh Xuan)

In [3]:
parse_address('Cuối đường kiều mai, dự án bệnh viện hạnh phúc  phường phú diễn quận bắc từ liêm hà nội')

AdministrativeUnit(province=Hà Nội, district=Bắc Từ Liêm, province_english=Ha Noi, district_english=Bac Tu Liem)

In [4]:
parse_address('03 Nguyễn Cao Luyện, An Hải Bắc, Sơn Trà, Đà Nẵng 550000, Việt Nam')

AdministrativeUnit(province=Đà Nẵng, district=Sơn Trà, province_english=Da Nang, district_english=Son Tra)

In [5]:
parse_address('3c60/10 chợ cầu xấng, xã phạm văn hai, huyện bình chánh HCM')

AdministrativeUnit(province=Hồ Chí Minh, district=Bình Chánh, province_english=Ho Chi Minh, district_english=Binh Chanh)

In [6]:
parse_address('ngõ 154 Đình Thôn, phường Mỹ Đình, Quận Nam Từ Liêm, HN')

AdministrativeUnit(province=Hà Nội, district=Nam Từ Liêm, province_english=Ha Noi, district_english=Nam Tu Liem)

In [7]:
parse_address('ngõ 154 Đình Thôn, phường Mỹ Đình, Quận Nam Từ Liêm, HN', level=1)

AdministrativeUnit(province=Hà Nội, district=None, province_english=Ha Noi, district_english=None)

# Check wrong or missing in bulk

In [15]:
df = pd.read_csv('../data/input/danhmuchanhchinhgso.gov.vn.csv')

In [16]:
df

,province,district,ward,ward_level
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Phường
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Phường
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Phường
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Phường
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Phường
...,...,...,...,...
10542,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Xã
10543,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Xã
10544,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Thị trấn
10545,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Xã


In [17]:
df['address'] = df['district'] + ', ' + df['province']

In [18]:
addresses = df['address'].tolist()

In [20]:
for address in addresses:
    try:
        unit = parse_address(address, level=2)
    except:
        print(address)
    if unit.province == None:
        print(address)
    elif unit.district == None:
        print(address)

In [22]:
df[df.ward.fillna('').str.contains('-')]

,province,district,ward,ward_level,address


In [52]:
df.ward_level.value_counts()

ward_level
Xã          8149
Phường      1775
Thị trấn     618
Huyện          5
Name: count, dtype: int64

In [27]:
def clean_ward(text):
    if not isinstance(text, str):
        return text
    c_text = re.sub(r'^Phường |^Thị trấn |^Xã |^Huyện', '', text, flags=re.IGNORECASE)
    return c_text

In [28]:
df['ward_clean'] = df['ward'].apply(clean_ward)

In [47]:
ward_count = df[['province', 'district', 'ward_clean']].value_counts().reset_index()

In [48]:
ward_count

,province,district,ward_clean,count
0,Tỉnh Cà Mau,Thành phố Cà Mau,Tân Thành,2
1,Thành phố Hà Nội,Huyện Gia Lâm,Yên Viên,2
2,Tỉnh Kiên Giang,Huyện Vĩnh Thuận,Vĩnh Thuận,2
3,Tỉnh Lâm Đồng,Huyện Đạ Huoai,Ma Đa Guôi,2
4,Tỉnh Sóc Trăng,Huyện Long Phú,Long Phú,2
...,...,...,...,...
10518,Tỉnh Hà Nam,Huyện Lý Nhân,Xuân Khê,1
10519,Tỉnh Hà Nam,Huyện Lý Nhân,Đạo Lý,1
10520,Tỉnh Hà Nam,Huyện Lý Nhân,Đức Lý,1
10521,Tỉnh Hà Nam,Huyện Thanh Liêm,Kiện Khê,1


In [49]:
df_duplicated_wards = ward_count[ward_count['count'] > 1]

In [50]:
df_duplicated_wards

,province,district,ward_clean,count
0,Tỉnh Cà Mau,Thành phố Cà Mau,Tân Thành,2
1,Thành phố Hà Nội,Huyện Gia Lâm,Yên Viên,2
2,Tỉnh Kiên Giang,Huyện Vĩnh Thuận,Vĩnh Thuận,2
3,Tỉnh Lâm Đồng,Huyện Đạ Huoai,Ma Đa Guôi,2
4,Tỉnh Sóc Trăng,Huyện Long Phú,Long Phú,2
5,Tỉnh Yên Bái,Huyện Yên Bình,Yên Bình,2
6,Tỉnh Đồng Tháp,Huyện Cao Lãnh,Mỹ Thọ,2
7,Tỉnh Đồng Tháp,Huyện Tháp Mười,Mỹ An,2
8,Tỉnh Sơn La,Huyện Mai Sơn,Hát Lót,2
9,Tỉnh Lai Châu,Huyện Mường Tè,Mường Tè,2


In [51]:
df[(df.province.isin(df_duplicated_wards.province)) & (df.district.isin(df_duplicated_wards.district)) & (df.ward_clean.isin(df_duplicated_wards.ward_clean))]

,province,district,ward,ward_level,address,ward_clean
176,Thành phố Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Thị trấn,"Huyện Gia Lâm, Thành phố Hà Nội",Yên Viên
178,Thành phố Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Xã,"Huyện Gia Lâm, Thành phố Hà Nội",Yên Viên
1479,Tỉnh Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Thị trấn,"Huyện Mường Tè, Tỉnh Lai Châu",Mường Tè
1484,Tỉnh Lai Châu,Huyện Mường Tè,Xã Mường Tè,Xã,"Huyện Mường Tè, Tỉnh Lai Châu",Mường Tè
1706,Tỉnh Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Thị trấn,"Huyện Mai Sơn, Tỉnh Sơn La",Hát Lót
1716,Tỉnh Sơn La,Huyện Mai Sơn,Xã Hát Lót,Xã,"Huyện Mai Sơn, Tỉnh Sơn La",Hát Lót
1882,Tỉnh Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Thị trấn,"Huyện Trạm Tấu, Tỉnh Yên Bái",Trạm Tấu
1887,Tỉnh Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Xã,"Huyện Trạm Tấu, Tỉnh Yên Bái",Trạm Tấu
1918,Tỉnh Yên Bái,Huyện Yên Bình,Thị trấn Yên Bình,Thị trấn,"Huyện Yên Bình, Tỉnh Yên Bái",Yên Bình
1937,Tỉnh Yên Bái,Huyện Yên Bình,Xã Yên Bình,Xã,"Huyện Yên Bình, Tỉnh Yên Bái",Yên Bình


In [32]:
df[(df.province=='Tỉnh Cà Mau') & (df.district=='Thành phố Cà Mau') & (df.ward_clean=='Tân Thành')]

,province,district,ward,ward_level,address,ward_clean
10456,Tỉnh Cà Mau,Thành phố Cà Mau,Phường Tân Thành,Phường,"Thành phố Cà Mau, Tỉnh Cà Mau",Tân Thành
10457,Tỉnh Cà Mau,Thành phố Cà Mau,Xã Tân Thành,Xã,"Thành phố Cà Mau, Tỉnh Cà Mau",Tân Thành


In [33]:
df[(df.province.str.contains('Hà Nội')) & (df.district.str.contains('Nam Từ Liêm')) & (df.ward_clean=='Mỹ Đình 2')]

,province,district,ward,ward_level,address,ward_clean
202,Thành phố Hà Nội,Quận Nam Từ Liêm,Phường Mỹ Đình 2,Phường,"Quận Nam Từ Liêm, Thành phố Hà Nội",Mỹ Đình 2
